<a href="https://colab.research.google.com/github/zxc741xb/Association-rules/blob/main/311112018_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# read dataset
df = pd.read_csv('GroceryStoreDataSet.csv',names=['items'], header=None)
print(df.head(10))

                           items
0             MILK,BREAD,BISCUIT
1  BREAD,MILK,BISCUIT,CORNFLAKES
2            BREAD,TEA,BOURNVITA
3           JAM,MAGGI,BREAD,MILK
4              MAGGI,TEA,BISCUIT
5            BREAD,TEA,BOURNVITA
6           MAGGI,TEA,CORNFLAKES
7        MAGGI,BREAD,TEA,BISCUIT
8            JAM,MAGGI,BREAD,TEA
9                     BREAD,MILK


In [ ]:
df = list(df['items'].apply(lambda x:x.split(',')))

In [ ]:
df = pd.DataFrame(df)
df

,0,1,2,3
0,MILK,BREAD,BISCUIT,None
1,BREAD,MILK,BISCUIT,CORNFLAKES
2,BREAD,TEA,BOURNVITA,None
3,JAM,MAGGI,BREAD,MILK
4,MAGGI,TEA,BISCUIT,None
5,BREAD,TEA,BOURNVITA,None
6,MAGGI,TEA,CORNFLAKES,None
7,MAGGI,BREAD,TEA,BISCUIT
8,JAM,MAGGI,BREAD,TEA
9,BREAD,MILK,None,None


In [ ]:
# items in transaction
items = set()
for col in df:
    items.update(df[col].unique())
print(items)

{'COFFEE', 'SUGER', None, 'TEA', 'BISCUIT', 'COCK', 'BOURNVITA', 'CORNFLAKES', 'JAM', 'BREAD', 'MILK', 'MAGGI'}


In [ ]:
# remove None values
items = set()
for col in df:
    items.update(df[col].dropna().unique())
print(items)

{'COFFEE', 'SUGER', 'TEA', 'BISCUIT', 'COCK', 'BOURNVITA', 'CORNFLAKES', 'JAM', 'BREAD', 'MILK', 'MAGGI'}


In [ ]:
# one hot encoding
itemset = set(items)
encoded_vals = []
for index, row in df.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset) # 沒有包含的
    commons = list(itemset.intersection(rowset)) # 有包含的
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
ohe_df = pd.DataFrame(encoded_vals)
print(ohe_df.head(10))

   COFFEE  SUGER  TEA  COCK  BOURNVITA  CORNFLAKES  JAM  MAGGI  MILK  BISCUIT  \
0       0      0    0     0          0           0    0      0     1        1   
1       0      0    0     0          0           1    0      0     1        1   
2       0      0    1     0          1           0    0      0     0        0   
3       0      0    0     0          0           0    1      1     1        0   
4       0      0    1     0          0           0    0      1     0        1   
5       0      0    1     0          1           0    0      0     0        0   
6       0      0    1     0          0           1    0      1     0        0   
7       0      0    1     0          0           0    0      1     0        1   
8       0      0    1     0          0           0    1      1     0        0   
9       0      0    0     0          0           0    0      0     1        0   

   BREAD  
0      1  
1      1  
2      1  
3      1  
4      0  
5      1  
6      0  
7      1  
8      1 

In [ ]:
# filtering items support > 0.2
freq_items = apriori(ohe_df, min_support=0.2, use_colnames=True)
print(freq_items.sort_values('support', ascending = False))

    support              itemsets
8      0.65               (BREAD)
0      0.40              (COFFEE)
2      0.35                 (TEA)
7      0.35             (BISCUIT)
1      0.30               (SUGER)
4      0.30          (CORNFLAKES)
5      0.25               (MAGGI)
6      0.25                (MILK)
3      0.20           (BOURNVITA)
9      0.20       (COFFEE, SUGER)
10     0.20  (COFFEE, CORNFLAKES)
11     0.20        (SUGER, BREAD)
12     0.20          (TEA, MAGGI)
13     0.20          (TEA, BREAD)
14     0.20         (MILK, BREAD)
15     0.20      (BISCUIT, BREAD)


/usr/local/lib/python3.9/dist-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [ ]:
# filtering items confidence > 0.6
rules = association_rules(freq_items, metric='confidence', min_threshold=0.6)
print(rules.sort_values('confidence', ascending = False))

    antecedents consequents  antecedent support  consequent support  support  \
3       (MAGGI)       (TEA)                0.25                0.35      0.2   
4        (MILK)     (BREAD)                0.25                0.65      0.2   
0       (SUGER)    (COFFEE)                0.30                0.40      0.2   
1  (CORNFLAKES)    (COFFEE)                0.30                0.40      0.2   
2       (SUGER)     (BREAD)                0.30                0.65      0.2   

   confidence      lift  leverage  conviction  
3    0.800000  2.285714    0.1125        3.25  
4    0.800000  1.230769    0.0375        1.75  
0    0.666667  1.666667    0.0800        1.80  
1    0.666667  1.666667    0.0800        1.80  
2    0.666667  1.025641    0.0050        1.05  


In [ ]:
print(ohe_df['MAGGI'].sum())
print(ohe_df['TEA'].sum())
print(ohe_df['MILK'].sum())
print(ohe_df['BREAD'].sum())

5
7
5
13
